## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-07-14-55-09 +0000,nypost,Pope Leo XIV’s former church and home hits mar...,https://nypost.com/2026/01/07/us-news/pope-leo...
1,2026-01-07-14-54-33 +0000,nyt,Venezuela Live Updates: U.S. Forces Board Oil ...,https://www.nytimes.com/live/2026/01/07/world/...
2,2026-01-07-14-52-03 +0000,startribune,US seizes 2 sanctioned oil tankers linked to V...,https://www.startribune.com/us-forces-board-ve...
3,2026-01-07-14-48-00 +0000,wsj,U.S. Forces Seize Fleeing Tanker Being Escorte...,https://www.wsj.com/world/u-s-forces-launch-op...
4,2026-01-07-14-48-00 +0000,wsj,Stocks Mixed While Oil Futures Decline,https://www.wsj.com/finance/stocks/global-mark...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,49
7,venezuela,45
12,oil,22
205,maduro,20
43,greenland,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
242,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...,159
108,2026-01-07-10-10-24 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,152
185,2026-01-07-00-32-44 +0000,nypost,Venezuela selling up to 50 million barrels of ...,https://nypost.com/2026/01/06/us-news/venezuel...,145
127,2026-01-07-09-07-00 +0000,wsj,Venezuela to Give U.S. Up to 50 Million Barrel...,https://www.wsj.com/politics/policy/venezuela-...,144
156,2026-01-07-03-19-29 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...,134


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
242,159,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...
108,103,2026-01-07-10-10-24 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...
120,69,2026-01-07-10-00-00 +0000,wsj,The welfare-fraud scandal that led Minnesota G...,https://www.wsj.com/politics/elections/minneso...
258,58,2026-01-06-21-51-00 +0000,wsj,Secretary of State Rubio told lawmakers that r...,https://www.wsj.com/politics/national-security...
311,57,2026-01-06-17-47-55 +0000,nypost,AI deepfakes of Nicolás Maduro flood social me...,https://nypost.com/2026/01/06/business/ai-deep...
85,44,2026-01-07-11-00-00 +0000,nypost,LA fire victims have just bills and broken pro...,https://nypost.com/2026/01/07/us-news/la-fire-...
213,41,2026-01-06-23-17-54 +0000,latimes,New California tool can stop brokers from sell...,https://www.latimes.com/california/story/2026-...
14,37,2026-01-07-14-28-30 +0000,cbc,"Carney to visit China next week to talk trade,...",https://www.cbc.ca/news/politics/carney-to-vis...
36,35,2026-01-07-13-30-05 +0000,nypost,"US forces seize sanctioned, Russia-flagged oil...",https://nypost.com/2026/01/07/us-news/us-force...
47,35,2026-01-07-13-07-01 +0000,nypost,Warner Bros. Discovery rejects Paramount Skyda...,https://nypost.com/2026/01/07/business/warner-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
